***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-895b  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import hashlib
import json

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  7 11:57 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir-project-bucket-313191645-201013310' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
#for b in blobs:
    #print(b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
title_anchor_pairs = parquetFile.select("anchor_text", "id").rdd.map(lambda x: (x[1], x[0])).mapValues(lambda x: [r['text'] for r in x])
doc_anchor_pairs = title_anchor_pairs.map(lambda x: (' '.join(x[1]), x[0]))

In [9]:
# Count number of wiki pages
total_number_of_docs_N = parquetFile.count()

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex
InvertedIndex.DIR_NAME = 'anchor_index/postings_gcp_anchor_index'

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
FILTERED = False
POSTING_LEN_THRESH = 300
TFIDF_ROUND_THRESH = 4
TFIDF_MUL = 10**TFIDF_ROUND_THRESH # 10000
DIR_NAME = "anchor_index/postings_gcp_anchor_index"

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

from functools import total_ordering

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  if FILTERED:
      set_of_toks = set(tokens)
      len_before = len(set_of_toks)
      len_after = len(set_of_toks - test_set_terms)
      if len_after == len_before:
        return []
  tf_counter = Counter()
  for tok in tokens:
    if tok not in all_stopwords:
      tf_counter[tok] += 1
  to_ret = []
  for item in tf_counter.items():
    to_ret.append((item[0],(id,item[1])))
  return to_ret

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=lambda x: x[0])

#start addition-------------------------
def get_posting_size(n):
    if POSTING_LEN_THRESH > n:
        return n
    return POSTING_LEN_THRESH

def our_sum(lst):
  summ = 0
  for i in lst:
    summ+=i
  return summ
#end addition------------------------------

def partition_postings_and_write(postings):
  temp = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  temp = temp.groupByKey()
  return temp.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))


In [14]:
#start addition-------------------------
doc_tf_vector = doc_anchor_pairs.map(lambda x: (x[1], word_count(x[0], x[1]))).filter(lambda x: len(x[1]) > 0)
doc_tf_vector_with_doc_len = doc_tf_vector.map(lambda x: (x[0], our_sum([i[1][1] for i in x[1]]),x[1]))
doc_tf_div_by_doc_len = doc_tf_vector_with_doc_len.map(lambda x: (x[0],[(term,(tup[0],tup[1]/x[1])) for term, tup in x[2]]))

flating = doc_tf_div_by_doc_len.map(lambda x: x[1]).flatMap(lambda x: x).groupByKey().mapValues(list)

add_df_for_value = flating.mapValues(lambda x: (len(x),x)).filter(lambda x: x[1][0] > 10)

posting_list_befor_trans = add_df_for_value.mapValues(lambda x: [(doc_id, tf*math.log(total_number_of_docs_N/x[0],2)) for doc_id, tf in x[1]])

doc_to_vec =  posting_list_befor_trans.map(lambda x: x[1]).flatMap(lambda x: x).groupByKey().mapValues(list)
doc_vec_size_rdd = doc_to_vec.mapValues(lambda x: our_sum([i**2 for i in x])**0.5)
#end addition------------------------------

In [15]:
doc_vec_size_dict  = dict(doc_vec_size_rdd.collect())

In [16]:
posting_list_div_by_doc_vec_size = posting_list_befor_trans.mapValues(lambda x: [(doc_id, tfidf/doc_vec_size_dict[doc_id]) for doc_id, tfidf in x])

In [17]:
rounded_posting_lists = posting_list_div_by_doc_vec_size.mapValues(lambda x: [(doc_id, int(tf_idf*10000)) for doc_id, tf_idf in x])

sorted_posting_list = rounded_posting_lists.mapValues(lambda x: sorted(x, key=lambda y: y[1], reverse=True))

# slicing posting list to POSTING_LEN_THRESH size (if posting list len is bigger)
final_posting_lists = sorted_posting_list.map(lambda x: (x[0], x[1][:get_posting_size(len(x[1]))]))
print('finish')

finish


In [18]:
_ = partition_postings_and_write(final_posting_lists).collect()

In [19]:
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=DIR_NAME):
  if not blob.name.endswith("pickle"):
    continue
  print("found " + blob.name)
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    print("wrote " + blob.name)
print("finish2")

found anchor_index/postings_gcp_anchor_index/0_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/0_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/100_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/100_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/101_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/101_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/102_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/102_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/103_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/103_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/104_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/104_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/105_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/105_posting_locs.pickle
found anchor_index/postings_gcp_anchor

wrote anchor_index/postings_gcp_anchor_index/47_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/48_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/48_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/49_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/49_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/4_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/4_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/50_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/50_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/51_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/51_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/52_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/52_posting_locs.pickle
found anchor_index/postings_gcp_anchor_index/53_posting_locs.pickle
wrote anchor_index/postings_gcp_anchor_index/53_po

In [20]:
df_dict_rdd = final_posting_lists.mapValues(lambda x: len(x))
df_dict = dict(df_dict_rdd.collect())

In [21]:
anchor_index = InvertedIndex()
anchor_index.df = df_dict
anchor_index.posting_locs = super_posting_locs

In [22]:
# write the global stats out
anchor_index.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/anchor_index/postings_tfidf_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 22.4 MiB/ 22.4 MiB]                                                
Operation completed over 1 objects/22.4 MiB.                                     


In [23]:
!gsutil ls -lh $index_dst

 22.35 MiB  2022-01-07T19:28:22Z  gs://ir-project-bucket-313191645-201013310/anchor_index/postings_tfidf_gcp/anchor_index.pkl
TOTAL: 1 objects, 23431906 bytes (22.35 MiB)


In [24]:
title_index.posting_locs['silicon']

NameError: name 'title_index' is not defined